![aiedge](https://ai-edge.be/assets/img/ai-edge.fe085446.png)

# CAR CLASSIFIER
This notebook will guide you through the process of testing a pretrained MobileNet car/no-car cassification network and exporting it from TensorFlow to TFLite Micro. In addition to that, we also quantize the model to 8-bits.
Afterwards, we can run this model on a micro-controller, such as the ESP32.

For more information, check out the TensorFlow documentation: https://www.tensorflow.org/lite.

In [ ]:
import os

!rm -rf sample_data

if not os.path.exists('dataset'):
  print('Fetching Data...')
  !wget -q -O embeddedai-data.tar.xz https://iiw.kuleuven.be/onderzoek/eavise/embedded-ai-workshop-data/at_download/file
  !tar -xf embeddedai-data.tar.xz
  print('Data downloaded!\n')
else:
  print('Data already downloaded!\n')

print('Installing dependencies...')
!apt-get -qq install xxd > /dev/null
!pip install tensorflow==2.4.1 > /dev/null
print('Dependencies installed!\n')

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

print('Tensorflow Version', tf.__version__)

## Loading the Model

Here we will load a model which includes architecture and weight data.  
This model has been pretrained on the coco dataset and is stored in the **model/** directory.

In [ ]:
model = tf.keras.models.load_model('./model', compile=True)
model.summary()

## Testing the Model
Before we quantize and convert the model to TFLite, we first want to run the original tensorflow model through a test dataset.  
These results can then be compared with the results from our quantized model on the ESP-EYE, in order to validate our model is working correctly.

### Testing Dataset
We supplied a testing dataset in the **dataset/** folder.  
The folder contains two subfolders **car/** and **no_car/**, which contain images with and without cars respectively.

In [ ]:
test_dataset = image_dataset_from_directory(
    './dataset',
    class_names=('no_car', 'car'),
    shuffle=True,
    batch_size=1,
    image_size=model.input_shape[1:3],
    interpolation='bilinear',
)

### Testing Accuracy
We can now use this dataset to evaluate our model.

In [ ]:
loss, accuracy = model.evaluate(test_dataset)

print(f'Test Accuracy = {100 * accuracy:.2f}%')
print(f'Test loss = {loss:.2f}')

### Visualize Test Examples
We can also use this dataset to visualize some results.

In [ ]:
# Create a matplotlib plot
plt.figure(figsize=(10, 10))

# Loop through 9 images
for i, (images, labels) in enumerate(test_dataset.take(9)):
    
    # Create a 3x3 grid of images and select image I
    ax = plt.subplot(3, 3, i + 1)
    
    # Plot the image
    plt.imshow(images[0].numpy().astype('uint8'))
    
    # Perform a prediction
    predicted_probability = model.predict(images[0:1])[0]
    
    # Interpret the probability into the correct class 'car' or 'no_car'
    predicted_class = ### TODO ###
    
    # Add ground truth and prediction as a title
    plt.title(
        f'Ground Truth = {test_dataset.class_names[labels[0]]}\n'
        f'Prediction = {predicted_class}'
    )
    
    # Disable plotting axis
    plt.axis('off')

## Exporting the Model
Now we need to export the model to TFLite.  
This time, we will also use TFLite to quantize the model to int8.

### Workaround a bug

Since Tensorflow 2.3 there is [a bug](https://github.com/tensorflow/tensorflow/issues/45256) with the reshape layer when this layer is converted to tensorflow lite.  
For some reason the converter produces a non supported sequence of `Shape → Strided Slice → Pack → Reshape` for each reshape layer instead of just using the shape layer.
Unfortunately, earlier Tensorflow versions that do not have this bug, have limited INT8 support which we want to use.

We work around this bug by removing the reshape layer in our tensorflow model.  
The only disadvantage is that the output will have a shape of `(None, 1, 1, 1)` instead of `(None, 1)`, so we need to take this into consideration.

Note that this fix removes the preprocessing layers too which would be removed by the converter anyway if kept.

In [ ]:
# ORIGINAL MODEL
if len(model.layers) == 4:
  # Select only the model part
  model_new = model.layers[-1]

  # Select the output of the layer just before the reshape layer
  x = model_new.layers[-3].output

  # Add an activation function on top
  x = tf.keras.layers.Activation(activation='sigmoid', name='predictions')(x)

  # Compose the new model
  model_new = tf.keras.Model(model_new.input, x)

# PRUNED MODEL
elif len(model.layers) == 91:
  # Create new sequential from the model layers until the reshape layer
  model_new = tf.keras.Sequential()
  for layer in model.layers[5:-2]:
    model_new.add(layer)

  # Add an activation function on top
  model_new.add(tf.keras.layers.Activation(activation='sigmoid', name='predictions'))

  # Build new model
  model_new.build((None, 96, 96, 3))
  
else:
  raise NotImplementedError('Unkown model variant')


# Print model summary
model = model_new
model.summary()

### Quantized TFLite Model
In addition to the the standard conversion, we are also going tell the Tensorflow Lite converter to apply Full integer quantization.  
_You can use the [following guide](https://www.tensorflow.org/lite/performance/post_training_quantization#integer_only) to fill in the blanks in the next code section._

In [ ]:
# Create a representative dataset for quantization
def representative_dataset():
    for images, _ in test_dataset.take(100):
        for image in images:
            res = np.expand_dims(image.numpy() / 255., axis=0).astype(np.float32)
            yield [res]

# Create the TFLite converter from our Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable full integer quantization
converter.representative_dataset = representative_dataset
converter.optimizations = ### TODO ###
converter.target_spec.supported_ops = ### TODO ###
converter.inference_input_type = ### TODO ###
converter.inference_output_type = ### TODO ###

# Convert our model
tflite_model = converter.convert()

# Save tflite model to a file
with open('carclassifier.tflite', 'wb') as f:
    f.write(tflite_model)

We have now saved our TFLite model to a file, which we can visualize with Netron.  

First, download the **carclassifier.tflite** file from google colab.  
Click on the folder icon in the left sidebar and the right-click the file and download it to your computer.

Then, <a href="https://netron.app" target="_blank">Click here</a> to open netron and select your file to visualize it.

### Convert to C++ data
Finally, we need to convert the TFLite model to a C++ array.

> 
> _Do not forget to download **carclassifier.cpp** to your computer, after running the cell below._
> 

In [ ]:
!xxd -i carclassifier.tflite > carclassifier.cpp
!cat carclassifier.cpp

## Compare Model Probabilities
As a basic sanity check, we will compare the output probabilities of our TensorFlow model with the quantized TFLite model.  
If all went well, the difference should be pretty small.

The advantage of comparing the outputs as opposed to looking at the testing accuracy, is that this code could theoretically be used for any network, regardless of its task.
We indeed simply look at the output tensors and compute the MSE between our floating point and quantized model.

_You can use the [following guide](https://www.tensorflow.org/lite/guide/inference#load_and_run_a_model_in_python) to fill in the blanks in the next code section (scroll down to the second code block)._

In [ ]:
# Load the TFLite model from the path './carclassifier.tflite'
interpreter = tf.lite.Interpreter('./carclassifier.tflite')

# Allocate memory
interpreter.allocate_tensors()

# Get input & output details
# These are necessary to know where we need to place the input in memory and where we get the output
input_details = ### TODO ###
output_details = ### TODO ###

# Arrays to store the output probabilities
tf_outputs = []
tflite_outputs = []

# Loop through 25 images
for image, label in test_dataset.take(25):
    
    # Convert the input image ([0,255] uint8) for our quantized model ([0,1] float32)
    tf_input = tf.cast(image / 255, tf.float32)
    
    # Predict using tensorflow model
    tf_output = model.predict(tf_input)
    
    # Append probability to list
    tf_outputs.append(tf_output)
    
    
    # Convert the input image ([0,255] uint8) for our quantized model ([-128,127] int8)
    # HINT: Use tf.cast()
    tflite_input = ### TODO ###

    # Predict using TFLite model
    interpreter.set_tensor(### TODO ###)
    interpreter.invoke()
    tflite_output = interpreter.get_tensor(### TODO ###)

    # Convert the model output ([-128,127] int8) to a probability ([0,1] float32)
    tflite_output = ### TODO ###

    # Append probability to list
    tflite_outputs.append(tflite_output)

    
# Compute the Mean Squared Error
tf_outputs = np.concatenate(tf_outputs)
tflite_outputs = np.concatenate(tflite_outputs)
mse = ((tf_outputs - tflite_outputs) ** 2).mean()

print(tf_outputs.shape, tflite_outputs.shape)
print("Mean squared error =", mse)

![eavise](https://gitlab.com/EAVISE/branding/logo/-/raw/master/twitter/header_500.png)